# Enter Stock Name

In [ ]:
stock = input('Please enter Ticker Symbol(e.g. AAPL) & Press Enter: ')

Please enter Ticker Symbol(e.g. AAPL) & Press Enter: BTC


# Getting Data & Calculating Algorithm

In [ ]:
#import libraries
import pandas as pd
import time
import pandas as pd
import numpy as np
import requests 
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
url = 'https://rest.coinapi.io/v1/ohlcv/'+stock+'/USD/latest?period_id=1DAY&limit=200'
headers = {'X-CoinAPI-Key' : 'DD609BB7-FE1D-425A-9E13-5333F2B1C0F4'}
response = requests.get(url, headers=headers)
j = response.json() 
df = pd.DataFrame.from_dict(j)
df['time_close'] = (pd.to_datetime(df['time_close']))
all_close = df['price_close'].values.astype(float)

In [ ]:
# model building
train=all_close[:-15].reshape(-1, 1)
test=all_close[-15:].reshape(-1, 1)
scaler=MinMaxScaler()
scaled_train=scaler.fit_transform(train)
scaled_test=scaler.transform(test)
 
n_input= len(train) - 1
n_features=1
 
train_generator=TimeseriesGenerator(scaled_train,
                                     scaled_train,
                                      n_input,
                                      batch_size=1)
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping()
 
custom_early_stopping = EarlyStopping(   
    monitor='loss', 
    patience=35, 
    min_delta=0.009, 
    mode='min' 
)
model=Sequential()
model.add(LSTM(100,activation='relu',input_shape=(n_input,n_features),return_sequences=True))
model.add(LSTM(50,activation='relu',return_sequences=True))
model.add(LSTM(10,activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam',loss='mse')
 
##
 
model.fit(train_generator,epochs=140, callbacks=[custom_early_stopping])
 
test_predictions = []
#Select last n_input values from the train data
first_eval_batch = scaled_train[-n_input:]
#reshape the data into LSTM required (#batch,#timesteps,#features)
current_batch = first_eval_batch.reshape((1, n_input, n_features))
for i in range(len(test)):
# get prediction, grab the exact number using the [0]
  pred = model.predict(current_batch)[0]
# Add this prediction to the list
  test_predictions.append(pred)
# The most critical part, update the (#batch,#timesteps,#features
# using np.append(
# current_batch[:        ,1:   ,:] ---------> read this as
# current_batch[no_change,1:end,no_change]
# (Do note the second part has the timesteps)
# [[pred]] need the double brackets as current_batch is a 3D array
# axis=1, remember we need to add to the second part i.e. 1st axis
  current_batch = np.append(current_batch[:,1:,:],[[pred]],axis=1)
 
 
actual_predictions = scaler.inverse_transform(test_predictions)
rows =list(range(len(train), len(train)+15)) # this
pred_df = pd.DataFrame(data=actual_predictions,index=rows)
 
future_train = all_close.reshape(-1,1)
scaled_future_train=scaler.fit_transform(future_train)
n_input= len(future_train) - 1
n_features=1
 
future_generator=TimeseriesGenerator(scaled_future_train,
                                     scaled_future_train,
                                      n_input,
                                      batch_size=1)
custom_early_stopping = EarlyStopping(   
    monitor='loss', 
    patience=35, 
    min_delta=0.009, 
    mode='min'
)
 
model=Sequential()
model.add(LSTM(100,activation='relu',input_shape=(n_input,n_features),return_sequences=True))
model.add(LSTM(50,activation='relu',return_sequences=True))
model.add(LSTM(10,activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam',loss='mse')
 
 
 
 
model.fit(future_generator,epochs=140, callbacks=[custom_early_stopping])
 
future_predictions = []
#Select last n_input values from the train data
first_eval_batch = scaled_future_train[-n_input:]
#reshape the data into LSTM required (#batch,#timesteps,#features)
current_batch = first_eval_batch.reshape((1, n_input, n_features))
for i in range(len(test)):
# get prediction, grab the exact number using the [0]
  pred = model.predict(current_batch)[0]
# Add this prediction to the list
  future_predictions.append(pred)
# The most critical part, update the (#batch,#timesteps,#features
# using np.append(
# current_batch[:        ,1:   ,:] ---------> read this as
# current_batch[no_change,1:end,no_change]
# (Do note the second part has the timesteps)
# [[pred]] need the double brackets as current_batch is a 3D array
# axis=1, remember we need to add to the second part i.e. 1st axis
  current_batch = np.append(current_batch[:,1:,:],[[pred]],axis=1)
 
 
future_actual_predictions = scaler.inverse_transform(future_predictions)
rows =list(range(len(future_train),len(future_train)+15))
future_pred_df = pd.DataFrame(data=future_actual_predictions,index=rows)

Epoch 1/140
1/1 [==============================] - 3s 3s/step - loss: 0.5923
Epoch 2/140
1/1 [==============================] - 0s 161ms/step - loss: 0.5510
Epoch 3/140
1/1 [==============================] - 0s 182ms/step - loss: 0.5104
Epoch 4/140
1/1 [==============================] - 0s 167ms/step - loss: 0.4750
Epoch 5/140
1/1 [==============================] - 0s 161ms/step - loss: 0.4416
Epoch 6/140
1/1 [==============================] - 0s 165ms/step - loss: 0.4031
Epoch 7/140
1/1 [==============================] - 0s 197ms/step - loss: 0.3561
Epoch 8/140
1/1 [==============================] - 0s 172ms/step - loss: 0.3001
Epoch 9/140
1/1 [==============================] - 0s 177ms/step - loss: 0.2309
Epoch 10/140
1/1 [==============================] - 0s 176ms/step - loss: 0.1430
Epoch 11/140
1/1 [==============================] - 0s 171ms/step - loss: 0.0407
Epoch 12/140
1/1 [==============================] - 0s 180ms/step - loss: 0.0523
Epoch 13/140
1/1 [======================

In [ ]:
fig = px.line(df,x=df.index, y='price_close',template='plotly_dark')
fig.add_trace(go.Scatter(x=pred_df.index, y=pred_df[0], mode="lines",name='Test_Prediction'))
fig.add_trace(go.Scatter(x=future_pred_df.index, y=future_pred_df[0], mode="lines",name='Future_Prediction'))
fig.update_layout(title=stock+' Stock Price LSTM prediction & test performance')
fig.update_yaxes(title='$')
fig.update_xaxes(title='Days')
#fig.update_layout(xaxis_rangeslider_visible=True)
fig.show()
#fig.to_html('pred.html')
fig.write_html(stock+' Pred.html')

# Visualize The Data

In [ ]:
fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['price_open'],
                high=df['price_high'],
                low=df['price_low'],
                close=df['price_close'])])
fig.update_layout(xaxis_rangeslider_visible=False)
fig.add_trace(go.Scatter(x=pred_df.index, y=pred_df[0], mode="lines",name='Test_Prediction'))
fig.add_trace(go.Scatter(x=future_pred_df.index, y=future_pred_df[0], mode="lines",name='Future_Prediction'))
fig.update_layout(title=stock+' Stock Price candlestick data Intraday')
fig.update_layout(template='plotly_dark')
fig.update_yaxes(title='$')
fig.update_xaxes(title='Days')
fig.show()